In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [3]:
dataset=dataset.drop("User ID",axis=1)

In [4]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)
dataset

,Age,EstimatedSalary,Purchased,Gender_Male
0,19,19000,0,1
1,35,20000,0,1
2,26,43000,0,0
3,27,57000,0,0
4,19,76000,0,1
...,...,...,...,...
395,46,41000,1,0
396,51,23000,1,1
397,50,20000,1,0
398,36,33000,0,1


In [5]:
#howmany class in Target_Value. Balanced or imbalanced data
dataset['Purchased'].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [6]:
independent=dataset[['Age','EstimatedSalary','Gender_Male']]
dependent=dataset[['Purchased']]

In [7]:
independent.shape

(400, 3)

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(independent, dependent, test_size=1/3,random_state=0)
from sklearn.preprocessing import StandardScaler
sd=StandardScaler()
X_train=sd.fit_transform(X_train)
X_test=sd.transform(X_test)

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
grid_param={'penalty':['l2'],'solver':['lbfgs','liblinear','newton-cg','newton-cholesky','saga','sag']}
grid=GridSearchCV(LogisticRegression(),grid_param, refit=True, verbose=3, n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train,Y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['lbfgs', 'liblinear', 'newton-cg',
                                    'newton-cholesky', 'saga', 'sag']},
             scoring='f1_weighted', verbose=3)

In [10]:
print("Confusion Matrix value for best parameter:{}".format(grid.best_params_))

Confusion Matrix value for best parameter:{'penalty': 'l2', 'solver': 'liblinear'}


In [11]:
res=grid.cv_results_
table=pd.DataFrame.from_dict(res)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.028254,0.006486,0.014226,0.002695,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
1,0.011760,0.008618,0.013197,0.002542,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.804764,0.811321,0.68205,0.92351,0.922185,0.828766,0.089565,1
2,0.012619,0.004067,0.011247,0.006090,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
3,0.059088,0.007239,0.012399,0.002050,l2,newton-cholesky,"{'penalty': 'l2', 'solver': 'newton-cholesky'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
4,0.006497,0.003437,0.011081,0.002751,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2
5,0.005369,0.004441,0.010242,0.005962,l2,sag,"{'penalty': 'l2', 'solver': 'sag'}",0.799620,0.811321,0.68205,0.92351,0.922185,0.827737,0.089864,2


In [12]:
y_pred=grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(Y_test,y_pred)
print(cm)

[[79  6]
 [10 39]]


In [13]:
#Classification confusion matrix report
from sklearn.metrics import classification_report
clf_report=classification_report(Y_test,y_pred)
print(clf_report)

              precision    recall  f1-score   support

           0       0.89      0.93      0.91        85
           1       0.87      0.80      0.83        49

    accuracy                           0.88       134
   macro avg       0.88      0.86      0.87       134
weighted avg       0.88      0.88      0.88       134



In [14]:
from sklearn.metrics import roc_auc_score
roc=roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])
roc

0.9507803121248499